In [1]:
# !pip install liana decoupler==1.8.0 kneed omnipath
# !pip install vitessce anywidget starlette uvicorn

In [2]:
from os.path import isdir
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

Load in data as in [bivariate.ipynb](https://github.com/saezlab/liana-py/blob/main/docs/source/notebooks/bivariate.ipynb).

In [3]:
adata_path = "data/kuppe_heart19.h5ad"

adata = sc.read(adata_path, backup_url='https://figshare.com/ndownloader/files/41501073?private_link=4744950f8768d5c8f68c')
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

Write to zarr.

In [4]:
zarr_filepath = "data/kuppe_heart19.processed.zarr"

if not isdir(zarr_filepath):
    adata = optimize_adata(
        adata,
        obs_cols=["celltype_niche"],
        obsm_keys=["spatial"],
        var_cols=["log1p_total_counts"],
        optimize_X=True,
    )
    adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

In [5]:
adata

AnnData object with n_obs × n_vars = 4113 × 17703
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'celltype_niche', 'molecular_niche'
    var: 'gene_ids', 'feature_types', 'genome', 'SYMBOL', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt', 'rps', 'mrp', 'rpl', 'duplicated'
    uns: 'spatial', 'log1p'
    obsm: 'compositions', 'mt', 'spatial'
    layers: 'counts'

Set up Vitessce config. Default gene view shows all genes.

In [6]:
vc = VitessceConfig(schema_version="1.0.15", name='Kuppe Heart', description='Heart dataset from Kuppe et al.')
dataset = vc.add_dataset(name='Heart').add_object(AnnDataWrapper(
        adata_path=zarr_filepath,
        obs_embedding_paths=["obsm/spatial"],
        obs_embedding_names=["Spatial"],
        obs_set_paths=["obs/celltype_niche"],
        obs_set_names=["Cell Type Niche"],
        obs_feature_matrix_path="X",
))

# Set default spot radius
[spatial_radius] = vc.add_coordination("embeddingObsRadius")
[spatial_radius_mode] = vc.add_coordination("embeddingObsRadiusMode")
spatial_radius.set_value(3.5)
spatial_radius_mode.set_value("manual")

# Add views
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="Spatial")
scatterplot.use_coordination(spatial_radius).use_coordination(spatial_radius_mode)
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(scatterplot | (cell_sets / genes))

vc.widget()

VitessceWidget(config={'version': '1.0.15', 'name': 'Kuppe Heart', 'description': 'Heart dataset from Kuppe et…

If wanting to filter by genes, a coordination layer can be added.

In [7]:
vc = VitessceConfig(schema_version="1.0.15", name='Kuppe Heart', description='Heart dataset from Kuppe et al.')
dataset = vc.add_dataset(name='Heart').add_object(AnnDataWrapper(
        adata_path=zarr_filepath,
        obs_embedding_paths=["obsm/spatial"],
        obs_embedding_names=["Spatial"],
        obs_set_paths=["obs/celltype_niche"],
        obs_set_names=["Cell Type Niche"],
        obs_feature_matrix_path="X",
))

# Set default spot radius
[spatial_radius] = vc.add_coordination("embeddingObsRadius")
[spatial_radius_mode] = vc.add_coordination("embeddingObsRadiusMode")
spatial_radius.set_value(3.5)
spatial_radius_mode.set_value("manual")

# Set genes to show
[featureFilter] = vc.add_coordination("featureFilter")
featureFilter.set_value(['VTN', 'ITGAV', 'ITGB5', 'TIMP1', 'CD63'])

# Add views
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="Spatial")
scatterplot.use_coordination(spatial_radius).use_coordination(spatial_radius_mode)
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset).use_coordination(featureFilter)
vc.layout(scatterplot | (cell_sets / genes))

vc.widget()

VitessceWidget(config={'version': '1.0.15', 'name': 'Kuppe Heart', 'description': 'Heart dataset from Kuppe et…